In [1]:
# Ver. 1.1.1

# 업데이트 내용 ----------------
# 다수의 라이브러리들 업데이트 및 변경된 api를 적용.
# 등치지도가 보여지지 않던 버그 수정.#
# -----------------------------

# 불편한 점이나 오류 발견 시 피드백 바람.

In [2]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
#matplotlib.rcParams['axes.unicode_minus'] = False
#pd.options.mode.chained_assignment = None  # default='warn'
#plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time
from dateutil.relativedelta import relativedelta
tqdm.tqdm.pandas()
import ipywidgets as widgets

In [3]:
search_city = widgets.RadioButtons(
    options=['서울', '인천', '경기'],
    layout={'width': 'max-content'}
)

search_city_box = widgets.Box(
    [
        widgets.Label(value='검색 지역 선택:'),
        search_city
    ]
)

search_range = widgets.IntText(
    value=12,
    layout={'width': 'max-content'}
)

search_range_box = widgets.Box(
    [
        widgets.Label(value='검색 범위(개월):'),
        search_range
    ]
)

estate_type = widgets.RadioButtons(
    options=['아파트', '연립다세대', '오피스텔'],
    layout={'width': 'max-content'}
)

estate_type_box = widgets.Box(
    [
        widgets.Label(value='주택 유형 선택:'),
        estate_type        
    ]
)

buyer_type = widgets.RadioButtons(
    options=['SH', 'LH', 'IH', 'GH', 'SH, LH, IH & GH', 'SH, LH, IH, GH를 제외한 거래사례', 'SH, LH, IH, GH, 일반분양 전체'],
    layout={'width': 'max-content'}
)

buyer_type_box = widgets.Box(
    [
        widgets.Label(value='매수자 유형 선택:'),
        buyer_type        
    ]
)

map_display_value_selection = widgets.RadioButtons(
    options=['최소', '최대', '평균', '분위'],
    layout={'width': 'max-content'}
)

map_display_value_selection_box = widgets.Box(
    [
        widgets.Label(value='지도 표시값 선택:'),
        map_display_value_selection        
    ]
)

area_search_range_options = widgets.RadioButtons(
    options=['~이상', '~이하', '~이상 & ~이하'],
    layout={'width': 'max-content'}
)

area_search_range_options_box = widgets.Box(
    [
        widgets.Label(value='전용면적 검색 범위:'),
        area_search_range_options        
    ]
)

# 검색값 입력 시작 -----------------------------------------------------------

In [4]:
display(search_city_box)
display(search_range_box)
display(estate_type_box)
display(buyer_type_box)
display(map_display_value_selection_box)
display(area_search_range_options_box)

Box(children=(Label(value='검색 지역 선택:'), RadioButtons(layout=Layout(width='max-content'), options=('서울', '인천', …

Box(children=(Label(value='검색 범위(개월):'), IntText(value=12, layout=Layout(width='max-content'))))

Box(children=(Label(value='주택 유형 선택:'), RadioButtons(layout=Layout(width='max-content'), options=('아파트', '연립다세…

Box(children=(Label(value='매수자 유형 선택:'), RadioButtons(layout=Layout(width='max-content'), options=('SH', 'LH',…

Box(children=(Label(value='지도 표시값 선택:'), RadioButtons(layout=Layout(width='max-content'), options=('최소', '최대',…

Box(children=(Label(value='전용면적 검색 범위:'), RadioButtons(layout=Layout(width='max-content'), options=('~이상', '~이…

In [5]:
search_city_box.close()
search_range_box.close()
estate_type_box.close()
buyer_type_box.close()
map_display_value_selection_box.close()
area_search_range_options_box.close()

print('검색 지역', search_city.value, '(으)로 설정됨.')
print('검색 범위', search_range.value, '개월로 설정됨.')
print('주택 유형', estate_type.value, '(으)로 설정됨.')
print('매수자 유형', buyer_type.value, '(으)로 설정됨.')
print('지도 표시값', map_display_value_selection.value, '(으)로 설정됨.')
print('전용면적 검색 범위', area_search_range_options.value, '(으)로 설정됨.')

검색 지역 서울 (으)로 설정됨.
검색 범위 12 개월로 설정됨.
주택 유형 연립다세대 (으)로 설정됨.
매수자 유형 SH (으)로 설정됨.
지도 표시값 평균 (으)로 설정됨.
전용면적 검색 범위 ~이하 (으)로 설정됨.


In [6]:
quantile_range = widgets.IntSlider(0, 0, 100, description='분위값 설정:')
if map_display_value_selection.value == '분위':
    display(quantile_range)

range_min = widgets.FloatText()
range_min_box = widgets.Box(
    [
        widgets.Label(value='전용면적 ~이상:'),
        range_min
    ]
)

range_max = widgets.FloatText()
range_max_box = widgets.Box(
    [
        widgets.Label(value='전용면적 ~이하:'),
        range_max
    ]
)

if area_search_range_options.value == '~이상':
    display(range_min_box)
elif area_search_range_options.value == '~이하':
    display(range_max_box)
elif area_search_range_options.value == '~이상 & ~이하':
    display(range_min_box)
    display(range_max_box)

Box(children=(Label(value='전용면적 ~이하:'), FloatText(value=0.0)))

In [7]:
if map_display_value_selection.value == '분위':
    quantile_range.close()
    print(quantile_range.value, '분위로 설정됨.')
if area_search_range_options.value == '~이상':
    range_min_box.close()
    range_max_box.close()
    print('전용면적', range_min.value, '이상으로 설정됨.')
elif area_search_range_options.value == '~이하':
    range_min_box.close()
    range_max_box.close()
    print('전용면적', range_max.value, '이하로 설정됨.')
elif area_search_range_options.value == '~이상 & ~이하':
    range_min_box.close()
    range_max_box.close()
    print('전용면적', range_min.value, '이상 &', range_max.value, '이하로 설정됨.')
    
    if range_min.value > range_max.value:
        raise ValueError('에러: 검색 범위 최소(이상) 값이 최대(이하) 값보다 클 수 없음.')

전용면적 40.0 이하로 설정됨.


In [8]:
timedelta_result = datetime.datetime.now() - relativedelta(months=search_range.value)

In [9]:
print('검색 시작 날짜:', str(timedelta_result.date()))

검색 시작 날짜: 2022-01-21


# 검색값 입력란 여기까지 --------------------------------------------------------

In [10]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [11]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [12]:
def data_prep(
    bdtype=estate_type.value,
    tradetype='매매',
    starting_year=timedelta_result.year,
    starting_date=str(timedelta_result.date()),
    city=search_city.value
):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    if (tradetype != '매매') & (tradetype != '전월세'):
        raise ValueError('두번째 변수는 매매 또는 전월세만 입력 가능')
    
    if city == '서울':
        basedir = './국토교통부_실거래가_공개시스템/{}/{}/'.format(bdtype, tradetype)
    elif city == '인천':
        basedir = './국토교통부_실거래가_공개시스템/인천/{}/{}/'.format(bdtype, tradetype)
    elif city == '경기':
        basedir = './국토교통부_실거래가_공개시스템/경기도/{}/{}/'.format(bdtype, tradetype)    
    
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm.tqdm_notebook(enumerate(filenames)):
        splitted_filename = f.split('실거래가_')       
        file_year = int(splitted_filename[-1][:4])
        
        if file_year < starting_year:
            continue
        
        try:
            df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        except:
            try:
                df = pd.read_csv(basedir + f, encoding='utf-8', header=15)
            except:
                df = pd.read_csv(basedir + f, header=15)
        
        if '건물명' in df.columns:
            df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
        elif '단지명' in df.columns:
            df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
            
        if '대지권면적(㎡)' in df.columns:
            df = df.drop(columns=['대지권면적(㎡)'])
                
            
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])
            
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    if tradetype == '전월세':
        concat_df = concat_df.rename(columns={'전월세구분':'거래구분', '보증금(만원)':'거래금액(만원)'})
        concat_df = concat_df.drop(columns=['월세(만원)'])
        concat_df = concat_df[concat_df['거래구분'] == '전세']
    elif tradetype == '매매':
        concat_df['거래구분'] = '매매'
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].astype('Int64')
    concat_df['계약일'] = concat_df['계약일'].astype('Int64')
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['번지', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    concat_df = concat_df[concat_df['계약날짜'] > pd.to_datetime(starting_date)]
    
    if '거래유형' in concat_df.columns:
        concat_df = concat_df.drop(columns=['거래유형'])
    if '중개사소재지' in concat_df.columns:
        concat_df = concat_df.drop(columns=['중개사소재지'])
    if '거래구분' in concat_df.columns:
        concat_df = concat_df.drop(columns=['거래구분'])
    concat_df = concat_df.drop(columns=['계약날짜기준_건물연식', '도로명', '시군구', '본번', '부번'])
    
    return concat_df

In [13]:
main_data_df = data_prep()
print(main_data_df.shape)
main_data_df.head()

C:\Users\max6296\AppData\Local\Temp\ipykernel_21888\2420055747.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, f in tqdm.tqdm_notebook(enumerate(filenames)):


0it [00:00, ?it/s]

(22430, 9)


,지번주소,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식
0,서울특별시 강남구 개포동 1172-2,(1172-2),42.15,50000,3,2012.0,2022-03-21,1186.239620,11.0
2,서울특별시 강남구 개포동 1216-4,(1216-4),29.97,55000,3,2012.0,2022-07-05,1835.168502,11.0
3,서울특별시 강남구 개포동 159-2,(159-2),57.55,110000,2,1987.0,2022-06-09,1911.381407,36.0
4,서울특별시 강남구 개포동 159-2,(159-2),121.23,199000,3,1987.0,2022-06-09,1641.507878,36.0
5,서울특별시 강남구 개포동 159-2,(159-2),41.45,78000,2,1987.0,2022-06-09,1881.785283,36.0


In [14]:
date_today = str(datetime.datetime.now().date())

if area_search_range_options.value == '~이상':
    area_range_name = str(range_min.value) + '이상'
elif area_search_range_options.value == '~이하':
    area_range_name = str(range_max.value) + '이하'
elif area_search_range_options.value == '~이상 & ~이하':
    area_range_name = str(range_min.value) + '이상_' + str(range_max.value) + '이하'

basedir = './구별_실거래가_지도/{}/{}_{}_전용면적_{}_최근_{}개월_{}'.format(
    date_today, search_city.value, estate_type.value, area_range_name, search_range.value, date_today
)

try:
    os.makedirs('./구별_실거래가_지도/{}/'.format(date_today))
except:
    pass

In [15]:
main_data_df.to_excel(basedir + '_전체_거래사례_자료.xlsx', index=False)

In [16]:
main_data_df['지번주소_건축년도'] = main_data_df['지번주소'] + '_' + main_data_df['건축년도'].astype('Int64').apply(str)

In [17]:
main_data_df['소유자'] = np.nan

In [18]:
def get_owner_name(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        owner_name = np.nan        
        addr_df = df[df['지번주소'] == addr]
        addr = addr.replace('-0', '')
        
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element_by_xpath('html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        time.sleep(1)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(1)
        
        elem_result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [19]:
def get_owner_name_update(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        owner_name = np.nan        
        addr_df = df[df['지번주소'] == addr]
        addr = addr.replace('-0', '')
        
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element(By.XPATH, 'html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        time.sleep(1)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        try:
            driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(1)
        
        elem_result = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [20]:
def name_filter(x):
    if pd.isna(x) == True:
        return x
    else:
        if x == '서***~':
            return 'SH'
        elif x == '한***~':
            return 'LH'
        elif x == '경***~':
            return 'GH'
        elif x == '인***~':
            return 'IH'
        else:
            return x

In [21]:
def buying_company_filter(df):
    df = df.copy()
    
    df['소유자'] = df['소유자'].apply(name_filter)
    
    return df

In [22]:
def export_data(df):
    df = df.copy()
    
    df = df[
        (df['소유자'] == 'SH')
        |(df['소유자'] == 'LH')
        |(df['소유자'] == 'GH')
        |(df['소유자'] == 'IH')
    ]
    
    df.to_excel(basedir + '_거래사례_베이스.xlsx', index=False)
    
    df['전용면적(㎡)_최소'] = df[['지번주소', '건축년도', '전용면적(㎡)']].groupby(['지번주소', '건축년도']).transform('min')['전용면적(㎡)']
    df['전용면적(㎡)_평균'] = df[['지번주소', '건축년도', '전용면적(㎡)']].groupby(['지번주소', '건축년도']).transform('mean')['전용면적(㎡)']
    df['전용면적(㎡)_최대'] = df[['지번주소', '건축년도', '전용면적(㎡)']].groupby(['지번주소', '건축년도']).transform('max')['전용면적(㎡)']
    df['전용면적(㎡)_합'] = df[['지번주소', '건축년도', '전용면적(㎡)']].groupby(['지번주소', '건축년도']).transform('sum')['전용면적(㎡)']
    
    df['거래금액(만원)_최소'] = df[['지번주소', '건축년도', '거래금액(만원)']].groupby(['지번주소', '건축년도']).transform('min')['거래금액(만원)']
    df['거래금액(만원)_평균'] = df[['지번주소', '건축년도', '거래금액(만원)']].groupby(['지번주소', '건축년도']).transform('mean')['거래금액(만원)']
    df['거래금액(만원)_최대'] = df[['지번주소', '건축년도', '거래금액(만원)']].groupby(['지번주소', '건축년도']).transform('max')['거래금액(만원)']
    df['거래금액(만원)_합'] = df[['지번주소', '건축년도', '거래금액(만원)']].groupby(['지번주소', '건축년도']).transform('sum')['거래금액(만원)']
    
    df['층_최대'] = df[['지번주소', '건축년도', '층']].groupby(['지번주소', '건축년도']).transform('max')['층']
    
    df['단가(만원/㎡)_최소'] = df[['지번주소', '건축년도', '단가(만원/㎡)']].groupby(['지번주소', '건축년도']).transform('min')['단가(만원/㎡)']
    df['단가(만원/㎡)_평균'] = df[['지번주소', '건축년도', '단가(만원/㎡)']].groupby(['지번주소', '건축년도']).transform('mean')['단가(만원/㎡)']
    df['단가(만원/㎡)_최대'] = df[['지번주소', '건축년도', '단가(만원/㎡)']].groupby(['지번주소', '건축년도']).transform('max')['단가(만원/㎡)']
    
    summary_df = df.drop(columns=['전용면적(㎡)', '거래금액(만원)', '층', '단가(만원/㎡)', '건물연식']).drop_duplicates()
    summary_df['구'] = summary_df['지번주소'].apply(lambda x: x.split(' ')[1])
    summary_df['동'] = summary_df['지번주소'].apply(lambda x: x.split(' ')[2])
    summary_df = summary_df.sort_values(['구', '동', '계약날짜', '지번주소']).reset_index(drop=True)
    summary_df = summary_df.drop(columns=['구', '동'])
    
    summary_df.to_excel(basedir + '_거래사례_요약.xlsx', index=False)

In [23]:
if buyer_type.value != 'SH, LH, IH, GH, 일반분양 전체':
    bulk_trade_idx = []

    print('일괄계약 사례 검색 중...')
    for addr in tqdm.tqdm_notebook(main_data_df['지번주소_건축년도'].unique()):
        addr_df = main_data_df[main_data_df['지번주소_건축년도'] == addr]

        if (addr_df.shape[0] >= 10) & (addr_df['계약날짜'].nunique() == 1):
            [bulk_trade_idx.append(idx) for idx in addr_df.index]
    
    bulk_trade_df = main_data_df.loc[bulk_trade_idx]
    
    print('일괄계약이 이루어진 지번주소 갯수:', bulk_trade_df['지번주소'].nunique())
    
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    
    print('소유자 정보를 가져오는 중...')
    time.sleep(1)
    driver = webdriver.Chrome(ChromeDriverManager().install())
    time.sleep(1)
    driver.get("http://www.iros.go.kr/PMainJ.jsp")
    time.sleep(1)
    driver.get("http://www.iros.go.kr/frontservlet?cmd=RISUWelcomeViewC")
    time.sleep(1)
    bulk_trade_df = get_owner_name_update(bulk_trade_df)
    
    bulk_trade_df = buying_company_filter(bulk_trade_df)
    
    export_data(bulk_trade_df)
    
    main_data_df.loc[bulk_trade_df.index, '소유자'] = bulk_trade_df['소유자']
    
    del bulk_trade_df
    
    main_data_df['소유자'].fillna('기타', inplace=True)
    
    if buyer_type.value == 'SH':
        main_data_df = main_data_df[main_data_df['소유자'] == 'SH']
    elif buyer_type.value == 'LH':
        main_data_df = main_data_df[main_data_df['소유자'] == 'LH']
    elif buyer_type.value == 'IH':
        main_data_df = main_data_df[main_data_df['소유자'] == 'IH']
    elif buyer_type.value == 'GH':
        main_data_df = main_data_df[main_data_df['소유자'] == 'GH']
    elif buyer_type.value == 'SH, LH, IH & GH':
        main_data_df = main_data_df[(main_data_df['소유자'] == 'SH')
                                    |(main_data_df['소유자'] == 'LH')
                                    |(main_data_df['소유자'] == 'IH')
                                    |(main_data_df['소유자'] == 'GH')
                                   ]
    elif buyer_type.value == 'SH, LH, IH, GH를 제외한 거래사례':
        main_data_df = main_data_df[
            (main_data_df['소유자'] != 'SH')
            &(main_data_df['소유자'] != 'LH')
            &(main_data_df['소유자'] != 'IH')
            &(main_data_df['소유자'] != 'GH')
        ]

일괄계약 사례 검색 중...


C:\Users\max6296\AppData\Local\Temp\ipykernel_21888\2805923717.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for addr in tqdm.tqdm_notebook(main_data_df['지번주소_건축년도'].unique()):


  0%|          | 0/13152 [00:00<?, ?it/s]

일괄계약이 이루어진 지번주소 갯수: 190
소유자 정보를 가져오는 중...


C:\Users\max6296\AppData\Local\Temp\ipykernel_21888\2805923717.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\max6296\AppData\Local\Temp\ipykernel_21888\685047273.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):


  0%|          | 0/190 [00:00<?, ?it/s]

In [24]:
if search_city.value != '서울':
    raise ValueError('검색 지역이 서울이 아닐 시 등치지도는 생성되지 않습니다.')
    quit()

In [25]:
def filter_and_generate_map_data(
    df,
    data_type=map_display_value_selection.value,
    qr_value=quantile_range.value / 100,
    area_range_type=area_search_range_options.value,
    area_range_min=range_min.value,
    area_range_max=range_max.value
):
    df = df.copy()
    
    ['~이상', '~이하', '~이상 & ~이하']
    if area_range_type == '~이상':
        df = df[df['전용면적(㎡)'] >= area_range_min]
    elif area_range_type == '~이하':
        df = df[df['전용면적(㎡)'] <= area_range_max]
    elif area_range_type == '~이상 & ~이하':
        df = df[
            (df['전용면적(㎡)'] >= area_range_min)
            &(df['전용면적(㎡)'] <= area_range_max)
        ]
    
    df['구'] = df['지번주소'].apply(lambda x: x.split(' ')[1])
    
    gu_data_df = pd.DataFrame()
    for i, gu in enumerate(df['구'].unique()):
        gu_df = df[df['구'] == gu]
        
        gu_data_df.loc[i, 'name'] = gu
        if data_type == '최소':
            gu_data_df.loc[i, 'value'] = gu_df['단가(만원/㎡)'].min()
        elif data_type == '최대':
            gu_data_df.loc[i, 'value'] = gu_df['단가(만원/㎡)'].max()
        elif data_type == '평균':
            gu_data_df.loc[i, 'value'] = gu_df['단가(만원/㎡)'].mean()
        elif data_type == '분위':
            gu_data_df.loc[i, 'value'] = gu_df['단가(만원/㎡)'].quantile(qr_value)
    
    return df, gu_data_df    

In [26]:
filtered_df, map_data_df = filter_and_generate_map_data(main_data_df)

In [27]:
print(map_data_df.shape)
map_data_df

(19, 2)


,name,value
0,강남구,1542.696318
1,강동구,1016.800241
2,강북구,731.187447
3,관악구,1285.711960
4,구로구,794.671201
5,금천구,735.499823
6,도봉구,860.251310
7,동대문구,925.267781
8,동작구,966.046659
9,마포구,1242.381555


In [28]:
print('지도 자료 구 갯수:', map_data_df.shape[0])

지도 자료 구 갯수: 19


In [29]:
filtered_df.to_excel(basedir + '_지도_베이스_자료_전체.xlsx', index=False)
map_data_df.to_excel(basedir + '_지도_자료.xlsx', index=False)

In [30]:
import folium

In [31]:
geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

In [32]:
color_list = ['블루그린', '블루퍼플', '그린블루', '오렌지레드', '퍼플블루', '퍼플블루그린', '퍼플레드', '레드퍼플',
              '옐로우그린', '옐로우그린블루', '옐로우오렌지브라운', '옐로우오렌지레드']

folium_color_names = ['BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu', 'YlGn', 'YlGnBu', 'YlOrBr',
                      'YlOrRd']

color_dict = {color_list[i]: folium_color_names[i] for i in range(len(color_list))}

In [33]:
map_color_options = widgets.RadioButtons(
    options=['화이트블루', '블루그린', '블루퍼플', '그린블루', '오렌지레드', '퍼플블루', '퍼플블루그린', '퍼플레드', '레드퍼플',
             '옐로우그린', '옐로우그린블루', '옐로우오렌지브라운', '옐로우오렌지레드'
            ]
)

map_color_options_box = widgets.Box(
    [
        widgets.Label(value='지도 표시 색깔:'),
        map_color_options        
    ]
)

display(map_color_options_box)

Box(children=(Label(value='지도 표시 색깔:'), RadioButtons(options=('화이트블루', '블루그린', '블루퍼플', '그린블루', '오렌지레드', '퍼플블루'…

In [35]:
m = folium.Map(
    location=[37.566345, 126.977893]#,
    #tiles='Stamen Terrain'
)

if map_color_options.value == '화이트블루':
    folium.Choropleth(
        geo_data=geo_json,
        name='choropleth',
        data=map_data_df,
        columns=['name', 'value'],
        key_on='feature.properties.name'
    ).add_to(m)
else:
    folium.Choropleth(
        geo_data=geo_json,
        name='choropleth',
        data=map_data_df,
        columns=['name', 'value'],
        key_on='feature.properties.name',
        fill_color=color_dict[map_color_options.value]
    ).add_to(m)
    
map_title = '{}_전용면적_{}_최근_{}개월_{}'.format(
    estate_type.value, area_range_name, search_range.value, date_today
)

print(map_title)
m

연립다세대_전용면적_40.0이하_최근_12개월_2023-01-21


In [ ]:
m.save(basedir + '_등치지도_{}.html'.format(map_color_options.value))